# Example of Usage of CONFETTI

In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn import preprocessing
from pathlib import Path
import warnings
import tensorflow as tf
tf.keras.utils.disable_interactive_logging()
# Suppress specific warning
warnings.filterwarnings("ignore", category=FutureWarning)

## Load Data

In [2]:
from sktime.datasets import load_UCR_UEA_dataset
dataset = 'RacketSports'

#Data will load with shape (instances, dimensions, timesteps)
X_train, y_train = load_UCR_UEA_dataset(dataset, split="train", return_type="numpy3d")
X_test, y_test = load_UCR_UEA_dataset(dataset, split="test", return_type="numpy3d")

#Reshape data to (instances, timesteps, dimensions)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[2],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])

#Encode
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#One Hot the labels for the CNN
y_train_encoded, y_test_encoded = keras.utils.to_categorical(y_train), keras.utils.to_categorical(y_test)


nb_classes = len(np.unique(np.concatenate([y_train,y_test])))
input_shape = X_train.shape[1:] #The input shape for our CNN should be (timesteps, dimensions)

## Model

### Train a Model

In [ ]:
#Train model yourself
from models.fcn import ClassifierFCN
#
model = ClassifierFCN(input_shape=input_shape, nb_classes=nb_classes, dataset_name=dataset, verbose=False)
#
model.fit(X_train, y_train_encoded, X_test, y_test_encoded)

#### or Load a Trained Model

In [3]:
#Load trained model
# Ensure the path is always relative to the project root
model_path = Path.cwd().parent  / "models" / "trained_models" / dataset / f"{dataset}_fcn.keras"
# Load the model
model = keras.models.load_model(str(model_path))

In [23]:
instance = X_test[2].reshape(1, X_test.shape[1], X_test.shape[2])

In [25]:
np.argmax(model.predict(instance))

1

In [24]:
model.predict(instance)

array([[0.0188832 , 0.95592797, 0.02182426, 0.00336454]], dtype=float32)

# Explainer

Firt, we need to obtain the class activation map weights.

In [ ]:
from confetti.CAM import class_activation_map as cam

training_weights = cam.compute_weights_cam(model, X_test, dataset=dataset, save_weights=False, data_type='training')

In [ ]:
cam.visualize_cam(training_weights, 0)

We are now ready to create our CONFETTI explainer.

In [ ]:
from confetti.explainer.confetti_explainer import CONFETTI

ce = CONFETTI(model_path, X_train, X_test[:1], y_test[:1], y_train, training_weights)

In [ ]:
#ce_directory = Path.cwd().parent/ "results" / dataset
#ce.counterfactual_generator(ce_directory, save_counterfactuals=True, optimization=True)

In [7]:
ce_directory = Path.cwd().parent/ "results" / dataset
ce.parallelized_counterfactual_generator(ce_directory,save_counterfactuals=False,processes=5, theta=0.51)

Naive approach started for instance 0
Naive approach finished for instance 0
Optimization stage started for instance 0
Optimization of CE for Instance 0 in Window 4

Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

Instance: 0 | Window: 4 | Time: 2.5109241008758545
Optimization of CE for Instance 0 in Window 6
Instance: 0 | Window: 6 | Time: 2.502371072769165
Optimization of CE for Instance 0 in Window 7
Instance: 0 | Window: 7 | Time: 2.7760350704193115
Optimization stage finished for instance 0


(                                            Solution Window Test Instance  \
 0  [[0.6661, -0.549034, 1.787509, 1.787509, 4.933...      7             0   
 
   NUN Instance  
 0           82  ,
                                             Solution Window Test Instance  \
 0  [[0.6661, -0.549034, 1.787509, 1.787509, 4.933...      7             0   
 
   NUN Instance  
 0           82  )

In [17]:
df = pd.read_csv('/Users/alan.paredes/Desktop/confetti/results/ArticularyWordRecognition/confetti_naive_ArticularyWordRecognition_fcn_alpha_0.1.csv')

In [18]:
df.head()

,Solution,Window,Test Instance,NUN Instance
0,-0.18454 0.33115 0.33115 0.60101 0.60101 -0.10...,20,0,192
1,-0.78598 -0.78598 0.067546 0.067546 0.067546 -...,48,1,88
2,-0.77122 -0.66555 -0.70345 -0.60467 -0.72872 -...,55,2,194
3,0.72769 0.80655 0.69736 -0.030628 0.61849 0.95...,42,3,234
4,2.5177 0.99877 -0.033487 -0.033487 1.972 1.573...,50,4,32


In [41]:
df_opt = pd.read_csv('/Users/alan.paredes/Desktop/confetti/results/RacketSports/confetti_optimized_RacketSports_fcn_alpha_0.1.csv')

In [42]:
df_opt.head()

,Solution,Window,Test Instance,NUN Instance
0,0.681357 15.742111 15.742111 23.714512 27.750542 5.342655 5.714323 3.182622 1.291853 1.883796 2.223444 -1.259692 -1.923176 -1.377562 -0.333237 3.580071 1.125178 -0.191598 0.348502 -0.019291 0.883803 1.214441 1.327363 1.081061 -0.625225 -0.625225 -0.625225 -0.625225 -0.625225 -0.625225 0.947844 -9.076906 -9.076906 1.095559 1.095559 -7.441843 0.001572 -4.185455 -6.681756 -13.759642 -6.212112 -3.039183 -2.064157 -1.892836 0.024322 -2.457798 -0.955009 -0.566428 -0.233073 -0.185125 -0.101404 -0.651285 1.13938 1.13938 1.13938 1.13938 1.13938 1.13938 1.13938 1.13938 4.848906 24.53091 -17.681627 16.627476 16.627476 4.212246 3.344312 1.868026 -1.019251 -1.008394 2.282011 1.470323 -3.23066 -0.158609 -0.74642 0.122126 -2.615353 -2.144778 -1.627654 -1.126312 -0.912041 0.095389 1.486379 1.486379 1.486379 1.486379 1.486379 1.486379 1.486379 1.486379 -0.324931 3.006948 -10.919827 -5.281468 -5.281468 -6.011232 -1.435558 0.439456 0.498051 -0.111862 -0.519358 0.65519 0.769715 0.751071 2.0215 2.25854 1.238468 0.19709 -0.250357 -0.226387 -0.412823 -0.860269 -0.380862 -0.380862 -0.380862 -0.380862 -0.380862 -0.380862 -0.380862 -0.380862 0.732428 -19.586441 -3.13479 -0.332922 -0.332922 0.524684 -0.66318 -0.732428 0.886903 0.527348 -0.311615 0.284981 0.727101 0.02397 0.580615 0.359555 0.364882 -0.085228 0.386189 0.367545 -0.247694 -0.540665 -0.407496 -0.407496 -0.407496 -0.407496 -0.407496 -0.407496 -0.407496 -0.407496 -0.524684 -13.191684 11.002392 3.904504 3.904504 2.263867 1.448875 1.155904 0.633883 0.386189 0.604586 0.082565 0.063921 0.090555 0.825646 0.396843 -0.039951 -0.090555 0.327595 0.609912 0.44212 0.20508 -0.045277 -0.045277 -0.045277 -0.045277 -0.045277 -0.045277 -0.045277 -0.045277,7,0,91
1,1.642246 -2.240663 1.433837 1.433837 -1.614643 -12.361575 -9.521825 10.832135 14.314162 18.846834 17.7817 29.018112 7.747449 14.293423 -1.457624 2.215545 3.317231 3.317231 2.158384 2.158384 2.158384 2.158384 2.158384 1.632807 1.809408 1.765316 2.060351 1.009896 0.14205 0.489353 1.557633 -2.063293 -1.103704 -1.103704 -2.854163 -5.077618 -3.07261 2.990696 -4.089018 12.69174 1.445267 17.95131 7.615059 -6.48757 2.848579 0.938852 0.968004 0.968004 1.850322 1.850322 1.850322 1.850322 1.850322 2.568867 2.582181 2.829877 2.48445 1.722229 1.962344 1.409304 -0.020815 -4.731666 -3.634325 -3.634325 -1.607593 13.829355 7.333056 -5.17188 5.701464 -13.111542 -14.131066 6.980147 3.983588 11.024697 9.132711 -2.892901 1.147554 1.147554 -2.521514 -6.985314 -2.521514 -5.195528 -5.195528 -3.040938 -3.040938 2.112386 4.298791 0.82268 0.668716 0.537099 -0.122515 -0.010653 1.566063 -0.359555 -1.225151 -0.359555 0.340912 -1.675261 -5.305438 -5.609063 0.050604 25.20083 -7.071254 -4.447832 -1.102636 -0.071911 2.226579 2.226579 1.55541 1.55541 1.55541 1.55541 1.55541 -0.090555 -0.21307 -0.047941 -0.351565 -0.372872 -0.069248 -0.151812 -0.191763 0.553982 0.631219 0.631219 0.010653 1.390281 2.442313 0.737754 3.880534 12.92002 -7.590612 1.427568 -0.567298 -2.125371 -0.838962 -0.625893 0.412823 0.412823 1.190528 1.190528 1.190528 1.190528 1.190528 0.138495 0.021307 -0.154476 -0.22905 -0.25302 -0.063921 0.191763 -0.263674 -0.404833 0.079901 0.079901 0.141159 0.447447 0.956151 0.508704 1.637974 5.097695 -15.165243 12.397999 4.256069 1.38229 1.214498 1.648628 1.38229 1.38229 0.716447 0.716447 0.716447 0.716447 0.716447 -0.378199 -0.287644 -0.19709 -0.306288 -0.460763 -0.585942 -0.711121,4,1,33
2,-14.082026 -0.402479 -9.510226 -9.510226 -9.510226 -9.510226 -9.510226 -14.371841 -7.267253 -15.86125 -0.798719 -21.063898 -14.487089 -5.523857 1.474461 4.583972 3.272555 0.319862 0.308687 -2.713633 -0.358973 -0.705989 -1.289857 -1.69408 -2.15303 -2.67547 -2.67547 -2.67547 -2.67547 -2.67547 13.529846 3.205859 -2.954328 -2.954328 -2.954328 -2.954328 -2.954328 0.681448 -20.756693 -12.541816 16.347343 0.358841 7.129751 2.735883 1.068159 -0.310497 2.573991 3.971836 6.173492 10.099062 1.575977 -1.474502 -2.874245 -0.884874 -0.681316 -0.843673 3.922317 

In [44]:
from confetti.explainer.utils import load_multivariate_ts_from_csv
X_sample, y_sample = load_multivariate_ts_from_csv('/Users/alan.paredes/Desktop/confetti/benchmark/data/RacketSports_samples.csv')

In [46]:
len(X_sample)

16

In [48]:
y_sample

array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])

In [47]:
df_opt['Test Instance'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [50]:
X_sample.shape[0]

16

In [54]:
X_sample.shape

(16, 30, 6)

In [52]:
covered_instances = df_opt["Test Instance"].unique()

In [53]:
len(covered_instances)

16

## Visualize the Counterfactuals

In [11]:
ce.visualize_counterfactuals(0, optimized=True)

KeyError: 'Precision'

In [ ]:
ce.visualize_marks(0)

# Metrics

In [ ]:
ce.naive_counterfactuals

In [ ]:
from benchmark.evaluations.evaluator import Evaluator
ev = Evaluator('confetti_naive')
cf_metrics, metrics = ev.evaluate_dataset(dataset)

In [ ]:
viz.tradeoff_sparsity_proximity(cf_metrics)